# Homework 2
- INFO 4871 _or_ INFO 5871 (indicate)
- Spring 2019
- _Aaron Barge_

## Objective (all students)
The object of this assignment is to implement a basic user-knn recommendation algorithm as a Python class. I am supplying six files to help you do this:

- This file (Jupyter notebook) acts a driver for your class. You will run this notebook _last_ after your implementation is complete to demonstrate that it works.
- The python file `User_KNN.py`, which contains a commented implementation skeleton of the `User_KNN` class. You will finish out this implementation. 
- The python file `test_User_KNN.py`, which contains an incomplete unit test for the `User_KNN` class.
- The Excel file `test_data.xlsx`. You will use this to compute by hand the missing correct values for the tests `test_similarities_hwk2` and `test_predictions_hwk2`. 
- The file `test_ratings.csv`, which contains the same ratings as in the `test_data.xlsx` file.
- The file `ratings_film_3.txt`, which contains movie ratings from the FilmTrust data set. 

Turn in the following in a zip file:

- Homework 2.html file computed from this worksheet after your implementation is complete.
- Homework 2.ipynb (this file) run after your implementation is complete.
- test_User_KNN.py file with the test functions complete and working with your User_KNN implementation
- User_KNN.py file with completed implementation
- test_data.xslx file showing the values needed for the test case computed

## Objective (INFO 5871 only)
In addition to the objectives above, you will implement a second user-knn recommendation algorithm named `User_KNN2`. This one will add the following features to the implementation described above:

- Default prediction. If there are fewer than `min_users` in the prediction neighborhood, the algorithm will return the item mean as the prediction, rather than computing a prediction.
- Shrinkage. The algorithm will implement similarity shrinkage as described in class. All similarities scores will be multiplied by $\frac{|P_u \cap P_v|}{\beta + |P_u \cap P_v|}$, where $\beta$ is supplied in the constructor.
- Generalized labels. The code in `User_KNN.py` assumes that the data frame given to `fit` has columns `'userId'`, `'movieId'` and `'rating'`. `User_KNN2` will accept data frames with arbitrary column names, and the mapping of labels to user, item, and rating is input to the constructor.
- You will write your own unit test for `User_KNN2`, which contains all of the same tests as `test_User_KNN` but with the correct values as computed in a second page of the `test_data.xslx` spreadsheet.

Turn in the following in a zip file:

- Everything from the "all students" objective above.
- User_KNN2.py implementation
- test_User_KNN2.py implementation working with your User_KNN2 implementation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import logging
from heapq import nlargest
%matplotlib inline


In [2]:
%load_ext autoreload
# %aimport User_KNN
# %autoreload 2

from User_KNN import User_KNN

In [3]:
import unittest
from test_User_KNN import test_User_KNN

%aimport test_User_KNN

In [4]:
suite = unittest.TestLoader().loadTestsFromTestCase(test_User_KNN.test_User_KNN)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_get_items (test_User_KNN.test_User_KNN) ... ok
test_get_overlap (test_User_KNN.test_User_KNN) ... ok
test_get_profile (test_User_KNN.test_User_KNN) ... ok
test_get_profile_length (test_User_KNN.test_User_KNN) ... ok
test_get_profile_mean (test_User_KNN.test_User_KNN) ... ok
test_get_rating (test_User_KNN.test_User_KNN) ... ok
test_get_users (test_User_KNN.test_User_KNN) ... ok
test_neighbors (test_User_KNN.test_User_KNN) ... ok
test_predict (test_User_KNN.test_User_KNN) ... ok
test_predict_hwk2 (test_User_KNN.test_User_KNN) ... ok
test_similarities (test_User_KNN.test_User_KNN) ... ok
test_similarities_hwk2 (test_User_KNN.test_User_KNN) ... ok

----------------------------------------------------------------------
Ran 12 tests in 0.519s

OK


<unittest.runner.TextTestResult run=12 errors=0 failures=0>

### Load the FilmTrust ratings
Should be 301 users

In [22]:
ratings = pd.read_csv('ratings_film_3.csv', sep='\s+', names=['userId','movieId','rating'])
len(ratings['userId'].unique())
ratings.head()

,userId,movieId,rating
0,750,205,0.5
1,750,214,0.5
2,750,220,0.5
3,750,256,0.5
4,750,250,0.5


In [31]:
ratings = pd.read_csv('jester_ratings_300.csv', sep=',', header=0, names=['userId','movieId','rating'])
len(ratings['userId'].unique())
# ratings.head()

269

In [32]:
type(ratings)
ratings.index

RangeIndex(start=0, stop=11349, step=1)

### Set up the algorithm
Note this is pretty slow. It could take 10-20 minutes or longer depending on your hardware. It prints out every 10 uesrs, so you can estimate how long it will take to process all 301 of them.

In [33]:
alg = User_KNN(20, sim_threshold=0)
alg.fit(ratings)

In [37]:
alg.predict_for_user(870, [2,4,9,11])

2     0.684336
4     0.684336
9     0.684336
11    0.684336
dtype: float64

In [45]:
alg.predict_for_user(871, [1,3,4,1])

1    0.887219
3    0.887219
4    0.887219
1    0.887219
dtype: float64